In [1]:
%pip install --upgrade supabase==2.22.0 langchain-community==0.3.31 langchain-openai==0.3.35 --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

C:\Users\leandro\AppData\Local\Temp\ipykernel_16400\3362794367.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [4]:
import os
import json
from langchain_core.documents import Document

SOURCE_DIRECTORY = "./books"
NUM_BOOKS = 17

def load_docs_from_json():
    langchain_documents = []
    
    for i in range(1, NUM_BOOKS + 1):
        book_id = f"{i:03d}"
        file_path = os.path.join(SOURCE_DIRECTORY, f"{book_id}.json")

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                book_data = json.load(f)
        except FileNotFoundError:
            print(f"Aviso: Arquivo {file_path} não encontrado. Pulando.")
            continue

        for page, paragraphs in book_data.items():
            for paragraph_text in paragraphs:
                if paragraph_text and len(paragraph_text.strip()) > 10:
                    metadata = {
                        "source": os.path.basename(file_path),
                        "book_id": book_id,
                        "page": int(page)
                    }
                    doc = Document(
                        page_content=paragraph_text.strip(),
                        metadata=metadata
                    )
                    langchain_documents.append(doc)
    
    return langchain_documents

documents = load_docs_from_json()

In [ ]:
vector_store = SupabaseVectorStore.from_documents(
    documents,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=100,
)

In [11]:
import os
from supabase.client import create_client
from langchain_openai import OpenAIEmbeddings

# Configurações do Supabase
supabase_url = os.environ["SUPABASE_URL"]
supabase_key = os.environ["SUPABASE_SERVICE_KEY"]
supabase = create_client(supabase_url, supabase_key)

# Inicializa embeddings
embeddings = OpenAIEmbeddings()

def similarity_search_with_score(query: str, k: int = 5):
    # Gera o embedding do texto da query
    query_embedding = embeddings.embed_query(query)

    # Chama a função RPC 'match_documents' no Supabase, passando o embedding e o número de resultados
    response = supabase.rpc(
        "match_documents",
        {
            "query_embedding": query_embedding,
            "match_count": k,
            "filter": {}  # pode passar filtros JSON se quiser
        }
    ).execute()

    results = response.data

    # Cada item em results tem id, content, metadata e similarity
    # Podemos montar uma lista de tuplas (Document, similarity)
    from langchain_core.documents import Document

    docs_with_scores = []
    for item in results:
        doc = Document(
            page_content=item["content"],
            metadata=item["metadata"]
        )
        score = item["similarity"]
        docs_with_scores.append((doc, score))

    return docs_with_scores

# Exemplo de uso
query = "me fale sobre a falta de vontade"
results = similarity_search_with_score(query, k=5)

for doc, score in results:
    print(f"Score: {score:.4f}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-----")


Score: 0.8666
Content: É no princípio de execução de todo propósito debem que, com frequência, a falta de vontade se faz presente; mas, sabendo que a causa desse mal está na preguiçafeita hábito, enfrentá-la-emos com resolução, avaliandoem todo o seu volume os prejuízos que ela nos ocasionae sem alimentar, nem por mais um instante, nenhum dospensamentos negativos que promove.
Metadata: {'page': 42, 'source': '004.json', 'book_id': '004'}
-----
Score: 0.8664
Content: Deficiências e Propensões do Ser Humano A antideficiência que aconselhamos aplicar nos casosde falta de vontade é a decisão. Para que seja efetiva, teráde ser praticada conscientemente, com responsabilidade – como toda antideficiência exige –, sobrepondo-se comempenho à apatia até triunfar no forcejo psicológico. Oser deve demonstrar que é capaz de contrapor à abuliaque o domina a decisão de combatê-la. Conseguirá,assim, ter vontade para tudo.
Metadata: {'page': 41, 'source': '004.json', 'book_id': '004'}
-----
Score: 0.852